### Part 1: Ideal VQE Simulation (20 Points)

In [1]:
!pip install "qiskit>=0.44.0,<0.45.0" "qiskit-algorithms==0.1.0" "qiskit-aer==0.12.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 10.0 MB/s eta 0:00:00


In [3]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator


# Hamiltion Definition
hamiltonian = SparsePauliOp(
    ["II", "IZ", "ZI", "ZZ", "XX"],
    coeffs=[
        -1.052 + 0.0j,
         0.398 + 0.0j,
        -0.398 + 0.0j,
        -0.011 + 0.0j,
         0.181 + 0.0j
    ]
)

print("H₂ Hamiltonian at 0.735 Å:")
print(hamiltonian)  # jsut for verification


ansatz = TwoLocal(2, rotation_blocks="ry", entanglement_blocks="cx",
                 entanglement="linear", reps=1)
optimizer = SPSA(maxiter=100)
estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)


result = vqe.compute_minimum_eigenvalue(hamiltonian)
print("\nVQE Results:")
print(f"Ground state energy: {result.eigenvalue.real:.6f} Ha")
print(f"Optimal parameters: {result.optimal_parameters}")

H₂ Hamiltonian at 0.735 Å:
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XX'],
              coeffs=[-1.052+0.j,  0.398+0.j, -0.398+0.j, -0.011+0.j,  0.181+0.j])

VQE Results:
Ground state energy: -1.857316 Ha
Optimal parameters: {ParameterVectorElement(θ[0]): np.float64(1.960744685097094), ParameterVectorElement(θ[1]): np.float64(1.3283872170115008), ParameterVectorElement(θ[2]): np.float64(1.169985219436643), ParameterVectorElement(θ[3]): np.float64(4.614426029295389)}


# **Explaination**

Fist I just constructed the Hamiltonian Matrix which was provided using SparsePauliOp.

Then I used TwoLocal to construct ansatz, which is a parameterized quantum circuit structure, we may think of this as a polynomial structure, which will later be optimised to fit the hamiltonian:

f(x) = a·x + b·x² + c·x³ (this is like the TwoLocal structure).

The Hamiltonian defines what f(x) should approximate (like a target curve).
The optimizer adjusts a, b, c to fit the curve.

The result part is calculated as above using defined optimizer and estimator.



Let's verify the result using numpy.

In [6]:
import numpy as np
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
II = np.kron(I, I)
IZ = np.kron(I, Z)
ZI = np.kron(Z, I)
ZZ = np.kron(Z, Z)
XX = np.kron(X, X)
H = -1.052 * II + 0.398 * IZ - 0.398 * ZI - 0.011 * ZZ + 0.181 * XX

print("Hamiltonian matrix:")
print(H)

Hamiltonian matrix:
[[-1.063  0.     0.     0.181]
 [ 0.    -1.837  0.181  0.   ]
 [ 0.     0.181 -0.245  0.   ]
 [ 0.181  0.     0.    -1.063]]


In [7]:
eigenvalues = np.linalg.eigvalsh(H)
ground_state_energy = eigenvalues[0]

print("\nExact ground state energy:", ground_state_energy)


Exact ground state energy: -1.8573191777730087


In [10]:
# error
relative_error = np.abs(result.eigenvalue.real - ground_state_energy) / np.abs(ground_state_energy)
print(f"Relative Error: {relative_error:.6e}")

Relative Error: 1.921017e-06


### Part 2: VQE on a Noisy Simulator (20 Points)

- **Define a Noise Model (10 pts)**:  
  Use `qiskit_aer.noise` to create a `NoiseModel`. Introduce **bit-flip** (Pauli $X$) error with probability **0.05** after each CNOT.

- **Run Noisy VQE (10 pts)**:
  - Use `qiskit_aer.primitives.Estimator`
  - Apply your noise model
  - Re-run VQE and report energy. Compare with ideal case.

In [20]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp


# Hamiltion Definition
hamiltonian = SparsePauliOp(
    ["II", "IZ", "ZI", "ZZ", "XX"],
    coeffs=[
        -1.052 + 0.0j,
         0.398 + 0.0j,
        -0.398 + 0.0j,
        -0.011 + 0.0j,
         0.181 + 0.0j
    ]
)
ansatz = TwoLocal(2, rotation_blocks="ry", entanglement_blocks="cx",
                 entanglement="linear", reps=1)
optimizer = SPSA(maxiter=100)


def create_noise_model():
    cnot_error = depolarizing_error(0.05, 2)  # 2-qubit error

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(cnot_error, ['cx'])
    return noise_model

noise_model = create_noise_model()

#Set up noisy estimator
noisy_estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "noise_model": noise_model
    },
    run_options={"shots": 1000}
)

noisy_vqe = VQE(noisy_estimator, ansatz, optimizer)
noisy_result = noisy_vqe.compute_minimum_eigenvalue(hamiltonian)

# 6. Compare results
print(f"Noisy VQE result: {noisy_result.eigenvalue.real:.6f} Ha")
print(f"Optimal parameters: {noisy_result.optimal_parameters}")

Noisy VQE result: -1.827772 Ha
Optimal parameters: {ParameterVectorElement(θ[0]): np.float64(3.3478875209734187), ParameterVectorElement(θ[1]): np.float64(3.1313567188583207), ParameterVectorElement(θ[2]): np.float64(-6.331857552672301), ParameterVectorElement(θ[3]): np.float64(-0.021817855981862876)}


Although we are running 1000 shots per run, this reduces shot noise but doesn’t eliminate it entirely. So, we expect the result to change after a new run.

In [21]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp


# Hamiltion Definition
hamiltonian = SparsePauliOp(
    ["II", "IZ", "ZI", "ZZ", "XX"],
    coeffs=[
        -1.052 + 0.0j,
         0.398 + 0.0j,
        -0.398 + 0.0j,
        -0.011 + 0.0j,
         0.181 + 0.0j
    ]
)
ansatz = TwoLocal(2, rotation_blocks="ry", entanglement_blocks="cx",
                 entanglement="linear", reps=1)
optimizer = SPSA(maxiter=100)


def create_noise_model():
    cnot_error = depolarizing_error(0.05, 2)  # 2-qubit error

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(cnot_error, ['cx'])
    return noise_model

noise_model = create_noise_model()

#Set up noisy estimator
noisy_estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "noise_model": noise_model
    },
    run_options={"shots": 1000}
)

noisy_vqe = VQE(noisy_estimator, ansatz, optimizer)
noisy_result = noisy_vqe.compute_minimum_eigenvalue(hamiltonian)

# 6. Compare results
print(f"Noisy VQE result: {noisy_result.eigenvalue.real:.6f} Ha")
print(f"Optimal parameters: {noisy_result.optimal_parameters}")

Noisy VQE result: -1.818214 Ha
Optimal parameters: {ParameterVectorElement(θ[0]): np.float64(7.184892118595146), ParameterVectorElement(θ[1]): np.float64(-1.3076429625021149), ParameterVectorElement(θ[2]): np.float64(4.056790281887561), ParameterVectorElement(θ[3]): np.float64(-4.880598945772195)}


And sure enough, we do observe a changed value.


In [23]:
# Error comparison with the ideal case
noisy1=-1.818214
noisy2=-1.827772
ideal= -1.857316
nosiy_whole=(noisy1+noisy2)/2
relative_error_1 = np.abs(nosiy_whole - ideal) / np.abs(ideal)
print(f"Relative Error: {relative_error_1:.6e}")


Relative Error: 1.847989e-02


In [26]:
# for more rhobust calculations

import numpy as np
from qiskit_algorithms import VQE

n_trials = 10  # Number of independent runs
energies = []

for _ in range(n_trials):
    noisy_vqe = VQE(noisy_estimator, ansatz, optimizer)
    noisy_result = noisy_vqe.compute_minimum_eigenvalue(hamiltonian)
    energies.append(noisy_result.eigenvalue.real)

print("Noisy VQE Results (10 trials):")
print(f"Mean energy: {np.mean(energies):.6f} ± {np.std(energies):.6f} Ha")
print(f"Individual energies: {[f'{e:.6f}' for e in energies]}")

Noisy VQE Results (10 trials):
Mean energy: -1.813413 ± 0.010380 Ha
Individual energies: ['-1.802858', '-1.806078', '-1.819314', '-1.800872', '-1.820148', '-1.814526', '-1.799216', '-1.826036', '-1.814000', '-1.831086']


# **Explaination**

This section just implements the previos circuit but with some noise, the noise intoduced is **bit-flip** (Pauli $X$) error with probability **0.05** after each CNOT, which jsut means that means that after every CNOT gate operation, there is a 5% chance that either the control or target qubit will flip its state (|0⟩ ↔ |1⟩) due to noise.



### Part 3: Building the QEC Components (30 Points)

- **Encoding Circuit (10 pts)**:  
  Write a function that takes a `QuantumCircuit` and a 1-qubit `QuantumRegister` input. Add gates to encode logical qubit into 3 physical qubits using bit-flip code.

- **Syndrome Measurement Circuit (10 pts)**:  
  Add gates for a full syndrome measurement cycle using 2 ancilla qubits and a classical register.

- **Correction Circuit (10 pts)**:  
  Add gates to apply corrective $X$ operations based on syndrome outcomes.


In [6]:
# Encoding Circuit Function

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def encode_bit_flip_code(qc, input_qubit):
    q = qc.qubits
    qc.cx(q[input_qubit], q[input_qubit+1])  # CNOT input -> ancilla1
    qc.cx(q[input_qubit], q[input_qubit+2])  # CNOT input -> ancilla2

### 3-Qubit Bit-Flip Code: Encoding

This function implements the encoding step of the **3-qubit bit-flip quantum error correction code**.

It transforms a single logical qubit in the state:  
$$
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle
$$  
into a **logical qubit** encoded across three physical qubits as:

$$
|\psi_L\rangle = \alpha|0_L\rangle + \beta|1_L\rangle
$$

where the logical basis states are:
- $$|0_L\rangle = |000\rangle$$  
- $$|1_L\rangle = |111\rangle$$

---

This is achieved by applying two **CNOT gates**:

- CNOT from the input qubit to the first ancilla
- CNOT from the input qubit to the second ancilla

These operations duplicate the value of the input qubit onto two ancilla qubits. After the CNOTs, the resulting 3-qubit entangled state is:

$$
|\psi_L\rangle = \alpha|000\rangle + \beta|111\rangle
$$

This encoded state enables detection and c


In [7]:
def add_syndrome_measurement(qc, logical_start, ancilla_start, syndrome_bits):
    q = qc.qubits
    qc.cx(q[logical_start], q[ancilla_start])
    qc.cx(q[logical_start+1], q[ancilla_start])
    qc.measure(q[ancilla_start], syndrome_bits[0])
    qc.cx(q[logical_start+1], q[ancilla_start+1])
    qc.cx(q[logical_start+2], q[ancilla_start+1])
    qc.measure(q[ancilla_start+1], syndrome_bits[1])
    qc.reset(q[ancilla_start])
    qc.reset(q[ancilla_start+1])

###Explainaiton:-

This function adds **syndrome measurements** to a 3-qubit bit-flip encoded state to detect **bit-flip errors**.

It uses two ancilla qubits to measure the **parity** between pairs of logical qubits:

- Logical qubits: `q[logical_start]`, `q[logical_start + 1]`, `q[logical_start + 2]`
- Ancilla qubits: `q[ancilla_start]`, `q[ancilla_start + 1]`

The procedure is:

1. **Measure parity between qubit 0 and 1**:
   - Apply CNOT from each to ancilla 0:
     ```python
     qc.cx(q[logical_start], q[ancilla_start])
     qc.cx(q[logical_start + 1], q[ancilla_start])
     ```
   - Measure ancilla 0 into classical bit `syndrome_bits[0]`

2. **Measure parity between qubit 1 and 2**:
   - Apply CNOT from each to ancilla 1:
     ```python
     qc.cx(q[logical_start + 1], q[ancilla_start + 1])
     qc.cx(q[logical_start + 2], q[ancilla_start + 1])
     ```
   - Measure ancilla 1 into classical bit `syndrome_bits[1]`

3. **Reset** both ancilla qubits for future reuse:
   ```python
   qc.reset(q[ancilla_start])
   qc.reset(q[ancilla_start + 1])


In [8]:
def add_correction_circuit(qc, logical_start, syndrome_bits):
    q = qc.qubits
    qc.x(q[logical_start]).c_if(syndrome_bits, 3)
    qc.x(q[logical_start+1]).c_if(syndrome_bits, 1)
    qc.x(q[logical_start+2]).c_if(syndrome_bits, 2)

###Explaination :-

This function adds **classically controlled correction gates** to a quantum circuit based on the **syndrome measurements** obtained from the bit-flip error detection step.

---

#### Inputs:
- `logical_start`: Index of the first logical qubit (assumes 3 logical qubits starting here)
- `syndrome_bits`: A 2-bit classical register holding the result of the syndrome measurement

---

### Error Syndromes

Based on the outcome of the syndrome bits (interpreted as a 2-bit binary number), we determine which qubit has been flipped:

| Syndrome (`syndrome_bits`) | Binary | Flipped Qubit |
|----------------------------|--------|----------------|
| 00                         | 0b00   | No error       |
| 01                         | 0b01   | Qubit 2        |
| 10                         | 0b10   | Qubit 1        |
| 11                         | 0b11   | Qubit 0        |

---

### Correction Logic

The function applies Pauli-X gates conditionally based on the syndrome:

- If syndrome = `11` (`0b11` = 3): apply `X` to qubit 0
  ```python
  qc.x(q[logical_start]).c_if(syndrome_bits, 3)


### Part 4: Integrated QEC-VQE and Analysis (30 Points)

- **Create Protected Ansatz (20 pts)**:  
  Wrap your ansatz with QEC logic:
  - Use logical and ancilla qubits
  - Apply **encoding circuit**
  - Insert translated `TwoLocal` ansatz gates (logical $\rightarrow$ physical)
  - Apply **syndrome measurement** and **correction**
  - Decode using inverse of the encoding circuit

- **Run Protected VQE (5 pts)**:  
  Execute with noisy estimator using your protected ansatz

- **Analyze and Compare (5 pts)**:  
  Print final energy from protected run.  
  Create a **table/bar chart** to compare:
  - Ideal (noise-free)
  - Noisy
  - Error-corrected
  Add a short **paragraph analysis**.

---

In [ ]:
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Operator
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit_algorithms import VQE
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp


def create_protected_ansatz():
    """Builds a QEC-protected ansatz for 2 logical qubits (8 physical qubits total)"""
    qr = QuantumRegister(8, 'q')
    cr = ClassicalRegister(2, 'syndrome')
    qc = QuantumCircuit(qr, cr)


    params = [Parameter(f'θ_{i}') for i in range(4)]


    encode_bit_flip_code(qc, 0)
    encode_bit_flip_code(qc, 3)


    for i in range(3):
        qc.ry(params[0], qr[i])
        qc.ry(params[1], qr[i+3])


    for i in range(3):
        qc.cx(qr[i], qr[i+3])

    # Error correction
    add_syndrome_measurement(qc, 0, 6, cr)
    add_correction_circuit(qc, 0, cr)
    add_syndrome_measurement(qc, 3, 6, cr)
    add_correction_circuit(qc, 3, cr)

    return qc


def expand_hamiltonian(hamiltonian):
    term_map = {
        'II': 'IIIIIIII',
        'IZ': 'IIIIIIZI',
        'ZI': 'IIIIZIII',
        'ZZ': 'IIIIZIZI',
        'XX': 'IIIIXXII'
    }

    new_terms = []
    new_coeffs = []
    for term, coeff in zip(hamiltonian.paulis, hamiltonian.coeffs):
        new_terms.append(term_map[str(term)])
        new_coeffs.append(coeff)

    return SparsePauliOp(new_terms, new_coeffs)


hamiltonian = SparsePauliOp(
    ["II", "IZ", "ZI", "ZZ", "XX"],
    coeffs=[
        -1.052 + 0.0j,
         0.398 + 0.0j,
        -0.398 + 0.0j,
        -0.011 + 0.0j,
         0.181 + 0.0j
    ]
)


def create_noise_model():
    cnot_error = depolarizing_error(0.05, 2)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(cnot_error, ['cx'])
    return noise_model

noise_model = create_noise_model()


noisy_estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "noise_model": noise_model
    },
    run_options={"shots": 1000}
)

protected_hamiltonian = expand_hamiltonian(hamiltonian)


protected_ansatz = create_protected_ansatz()
optimizer = SPSA(maxiter=100)
protected_vqe = VQE(noisy_estimator, protected_ansatz, optimizer)
protected_result = protected_vqe.compute_minimum_eigenvalue(protected_hamiltonian)


ideal_energy = -1.857316
noisy_energy = -1.813413
protected_energy = protected_result.eigenvalue.real

print(f"\nProtected VQE energy: {protected_energy:.6f} Ha")
print(f"Improvement over noisy: {noisy_energy - protected_energy:.6f} Ha")